## Connect To Elastic Search

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'FQt-ffZfTpeh0Snf3pUAQw',
 'name': 'ae8b5b4be42b',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


### Index Documents

In [2]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the d`ocument_ids` list.

In [3]:
import json
from tqdm import tqdm

document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='my_index', body=document)
    document_ids.append(response['_id'])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 10.80it/s]


In [4]:
document_ids

['cCorNJoBqbZEoMTBrcoi', 'cSorNJoBqbZEoMTBrsoR', 'ciorNJoBqbZEoMTBrsoh']

### Update Api

### 1. If documents exists in the index

#### 1.1 Update an existing field

In [5]:
from pprint import pprint

response = es.update(
    index='my_index',
    id=document_ids[0],
    script={
        "source":"ctx._source.title = params.title",
        "params":{
            "title":"New Title"
        }
    }
)

pprint(response.body)


{'_id': 'cCorNJoBqbZEoMTBrcoi',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'updated'}


In [6]:
response = es.get(index='my_index', id=document_ids[0])
pprint(response.body)


{'_id': 'cCorNJoBqbZEoMTBrcoi',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_version': 2,
 'found': True}


#### 1.2 Add a new field

To add a new field, you can either use the `script` argument or the `doc` argument.

#### 1.2.1 Method 1 (Script)

In [7]:
response = es.update(
    index='my_index',
    id=document_ids[0],
    script={
        "source":"ctx._source.new_field = 'dummy_value'",
    },
)
pprint(response.body)

{'_id': 'cCorNJoBqbZEoMTBrcoi',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 4,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 3,
 'result': 'updated'}


In [8]:
response = es.get(index='my_index', id=document_ids[0])
pprint(response.body)

{'_id': 'cCorNJoBqbZEoMTBrcoi',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 4,
 '_source': {'created_on': '2024-09-22',
             'new_field': 'dummy_value',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_version': 3,
 'found': True}


#### 1.2.2 Method 2 (doc)